In [1]:
import sys
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime
import numpy as np

# Add path to Helpers to import model_utils
sys.path.append(os.path.abspath('../Helpers'))
import model_utils as mu

In [2]:
# --- 1. Load Data ---
BASE_PATH = '../data_recordings'
print(f"Loading manifest from: {BASE_PATH}")
df = mu.load_manifest(BASE_PATH)

# Optional: Sample data for quick debugging (uncomment next line)
# df = df.head(50)

Loading manifest from: ../data_recordings


In [3]:
# Load and process audio into spectrograms
X, y, speakers = mu.load_and_process_data(df, BASE_PATH)

Loading 11225 audio files...
Successfully loaded: 11225, Failed: 0
Generating Mel-Spectrograms...


In [4]:
# --- 2. Split Data ---
X_train, X_val, X_test, y_train, y_val, y_test, df_train, df_val, df_test = mu.split_dataset(X, y, speakers, df)

print(f"Train shape: {X_train.shape}")
print(f"Val shape:   {X_val.shape}")
print(f"Test shape:  {X_test.shape}")

Train shape: (7859, 128, 130, 1)
Val shape:   (1681, 128, 130, 1)
Test shape:  (1685, 128, 130, 1)


In [5]:
# --- 3. Define Experiments ---
# (Experiment Name, Optimizer, Dropout, Base LR, lr_schedule, weight_decay, use_plateau)
experiments = [
    # --- your existing ones (unchanged behavior) ---
    ("Baseline_Adam",   "adam",  0.0, 0.001,  None,     0.0,   False),
    ("Baseline_SGD",    "sgd",   0.0, 0.01,   None,     0.0,   False),
    ("Dropout_0.2",     "adam",  0.2, 0.001,  None,     0.0,   False),
    ("Low_LR_Adam",     "adam",  0.0, 0.0001, None,     0.0,   False),

    # --- up to 4 new ones ---
    ("Adam_Plateau",        "adam",  0.2, 0.001, None,     0.0,   True),
    ("SGD_Plateau",         "sgd",   0.0, 0.01,  None,     0.0,   True),
    ("AdamW_Cosine",        "adamw", 0.2, 0.001, "cosine",  1e-4,  False),
    ("AdamW_Plateau",       "adamw", 0.2, 0.001, None,     1e-4,  True),
]
results_summary = []
# Create a directory for results if it doesn't exist
os.makedirs("results", exist_ok=True)

In [6]:
import tensorflow as tf
import keras
import sys

print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("keras package:", keras.__version__)
print("keras module file:", keras.__file__)
print("tf.keras module:", tf.keras.__name__)

Python: 3.11.0 (main, Oct 24 2022, 18:26:48) [MSC v.1933 64 bit (AMD64)]
TensorFlow: 2.20.0
keras package: 3.12.0
keras module file: c:\Users\adamf\Desktop\student_debil\semestr7\WUT-IML-2025-2026\.venv\Lib\site-packages\keras\__init__.py
tf.keras module: tensorflow.keras


In [7]:
# --- 4. Run Experiments Loop ---
batch_size = 32
epochs = 15
steps_per_epoch = int(tf.math.ceil(len(X_train) / batch_size))

for exp_name, opt, drop, lr, lr_sched, wd, use_plateau in experiments:
    print(f"\n" + "="*40)
    print(f"Running Experiment: {exp_name}")
    print(f"Params: Opt={opt}, Dropout={drop}, LR={lr}, Sched={lr_sched}, WD={wd}, Plateau={use_plateau}")
    print("="*40)
    model = mu.build_model(
        input_shape=X_train.shape[1:],
        optimizer_name=opt,
        dropout_rate=drop,
        learning_rate=lr,
        lr_schedule=lr_sched,
        weight_decay=wd,
        steps_per_epoch=steps_per_epoch,
        total_epochs=epochs,
        adapt_data=X_train
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.CSVLogger(f"results/logs_{exp_name}.csv"),
        *mu.make_lr_callbacks(use_plateau=use_plateau),
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluate on Test Set
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {acc:.4f}")

    # Record results
    results_summary.append({
        "Experiment": exp_name,
        "Test_Accuracy": acc,
        "Test_Loss": loss,
        "Optimizer": opt,
        "Dropout": drop,
        "Learning_Rate": lr
    })

    # Save Model
    model_save_path = f"results/model_{exp_name}.keras"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # --- IMPORTANT FOR EDA ---
    # Save predictions with metadata for Error Analysis
    print("Generating predictions for analysis...")
    preds_prob = model.predict(X_test, verbose=0).flatten()

    # Create DataFrame with true labels, predictions, AND metadata (filenames)
    # We use df_test which we got from the split function
    pred_df = df_test.copy()
    pred_df["true_label"] = y_test
    pred_df["pred_prob"] = preds_prob
    pred_df["pred_label"] = (preds_prob > 0.5).astype(int)

    # Save to CSV
    pred_df.to_csv(f"results/predictions_{exp_name}.csv", index=False)

    if drop > 0:
        print("Generating Monte Carlo Dropout predictions...")
        mc_mean, mc_std = mu.mc_dropout_predict(model, X_test, n_passes=15)

        mc_df = df_test.copy()
        mc_df["true_label"] = y_test
        mc_df["pred_prob_mean"] = mc_mean
        mc_df["pred_prob_std"] = mc_std
        mc_df["pred_label"] = (mc_mean > 0.5).astype(int)

        print("\n--- MC Dropout Uncertainty Summary ---")
        print(f"Mean uncertainty (std): {mc_std.mean():.4f}")
        print(f"Median uncertainty:     {np.median(mc_std):.4f}")
        print(f"Max uncertainty:        {mc_std.max():.4f}")

        correct_mask = (mc_df["pred_label"].values == y_test)

        mean_unc_correct = mc_std[correct_mask].mean()
        mean_unc_wrong   = mc_std[~correct_mask].mean()

        print("\n--- Uncertainty vs Correctness ---")
        print(f"Mean uncertainty (correct preds): {mean_unc_correct:.4f}")
        print(f"Mean uncertainty (wrong preds):   {mean_unc_wrong:.4f}")


        mc_df.to_csv(f"results/predictions_mc_{exp_name}.csv", index=False)


Running Experiment: Baseline_Adam
Params: Opt=adam, Dropout=0.0, LR=0.001, Sched=None, WD=0.0, Plateau=False
Epoch 1/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 176s 691ms/step - accuracy: 0.6712 - loss: 0.5852 - val_accuracy: 0.7537 - val_loss: 0.4675
Epoch 2/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 161s 653ms/step - accuracy: 0.8527 - loss: 0.3348 - val_accuracy: 0.8828 - val_loss: 0.2870
Epoch 3/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 164s 666ms/step - accuracy: 0.8963 - loss: 0.2452 - val_accuracy: 0.9078 - val_loss: 0.2441
Epoch 4/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 159s 646ms/step - accuracy: 0.9197 - loss: 0.1937 - val_accuracy: 0.8626 - val_loss: 0.3448
Epoch 5/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 157s 639ms/step - accuracy: 0.9357 - loss: 0.1526 - val_accuracy: 0.9072 - val_loss: 0.2456
Epoch 6/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 161s 654ms/step - accuracy: 0.9466 - loss: 0.1356 - val_accuracy: 0.9227 - val_loss: 0.2059
Epoch 7/15
246/246 ━━━━━━━━━━━━━━━━━━━━ 167s 679ms/step - accuracy: 0.9597 - loss: 0.1066 - val_ac

In [8]:
# --- 5. Summary ---
summary_df = pd.DataFrame(results_summary)

# Add a timestamp column with the current date and time
summary_df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("\n=== Final Experiments Summary ===")
print(summary_df)

summary_path = "results/experiments_summary.csv"

# Check if file exists to determine whether to write the header
# Note: If an old file exists without a 'timestamp' column, appending might cause
# structure issues. It is best to delete the old file or manually add a column header to it.
write_header = not os.path.exists(summary_path)

# Save to CSV with mode='a' (append)
summary_df.to_csv(summary_path, mode='a', header=write_header, index=False)

print("\nAll experiments completed.")


=== Final Experiments Summary ===
      Experiment  Test_Accuracy  Test_Loss Optimizer  Dropout  Learning_Rate  \
0  Baseline_Adam       0.944807   0.157262      adam      0.0         0.0010   
1   Baseline_SGD       0.954896   0.144329       sgd      0.0         0.0100   
2    Dropout_0.2       0.959644   0.106261      adam      0.2         0.0010   
3    Low_LR_Adam       0.925223   0.183353      adam      0.0         0.0001   
4   Adam_Plateau       0.957270   0.115072      adam      0.2         0.0010   
5    SGD_Plateau       0.958457   0.149510       sgd      0.0         0.0100   
6   AdamW_Cosine       0.946588   0.126059     adamw      0.2         0.0010   
7  AdamW_Plateau       0.964392   0.102901     adamw      0.2         0.0010   

             timestamp  
0  2025-12-17 21:41:06  
1  2025-12-17 21:41:06  
2  2025-12-17 21:41:06  
3  2025-12-17 21:41:06  
4  2025-12-17 21:41:06  
5  2025-12-17 21:41:06  
6  2025-12-17 21:41:06  
7  2025-12-17 21:41:06  

All experiments co